# SPARC Containerization and Deployment

## 1.0 Introduction
This notebook covers packaging SPARC services as portable containers and preparing both local Podman validation and HiPerGator production deployment artifacts.

### 1.1 Objectives
1. **Containerize**: Build images for MAS backend, Unity Linux server runtime, and Signaling Server.
2. **Orchestrate**: Validate local Podman pod networking for server-side rendering (Pixel Streaming).
3. **Deploy**: Generate production SLURM artifacts for HiPerGator-compatible backend workflows.

### 1.2 Introduction Diagram
![Introduction](./images/notebook_2_-_section_1.png)

Introduction: This section defines the containerization objectives for the SPARC stack. We now support server-side rendering for thin clients by introducing a Unity Linux server runtime and a signaling container in addition to the backend services.

## 2.0 Containerization (Docker/Podman -> Apptainer)
We develop with Docker/Podman and deploy with Apptainer on HPC when needed.

![notebook 2 - section 2.png](images/notebook_2_-_section_2.png)

Container Build Strategy: The flow uses secure, minimal runtime images. Build steps compile dependencies in dedicated stages, then copy only required artifacts into runtime images.

This notebook provides image definitions for three build targets:
1. **MAS Backend** (`Dockerfile.mas`)
2. **Unity Linux Server Build** (`Dockerfile.unity-server`)
3. **WebRTC Signaling Server** (`Dockerfile.signaling`)

### 2.1 Dockerfile Definition

This section includes reference Dockerfiles for the MAS backend, Unity Linux server rendering runtime, and the Unity Render Streaming signaling server.

The Unity runtime image should use an NVIDIA OpenGL runtime base and launch the Linux player with Render Streaming-compatible flags.
The signaling image should be a lightweight Node.js runtime built from the Unity Render Streaming signaling package.

In [ ]:
# 2.2-2.4 Dockerfiles for MAS, Unity Linux Server, and Signaling
def create_dockerfiles():
    from pathlib import Path

    unity_build_dir = Path("artifacts/unity/LinuxServer")
    signaling_dir = Path("artifacts/signaling")

    requirements = """
fastapi
uvicorn[standard]
pydantic>=2.5.0
numpy>=1.24.0
aiofiles
websockets
python-multipart
transformers>=4.36.0
accelerate>=0.25.0
tokenizers>=0.15.0
bitsandbytes>=0.41.0
peft>=0.7.0
langchain>=0.1.0
langchain-community>=0.0.13
langchain-openai>=0.0.5
langchain-chroma>=0.1.0
langgraph>=0.0.26
nvidia-riva-client>=2.14.0
nemoguardrails>=0.5.0
chromadb>=0.4.22
presidio-analyzer>=2.2.33
presidio-anonymizer>=2.2.33
firebase-admin>=6.2.0
python-jose[cryptography]
python-dotenv
grpcio
grpcio-tools
""".strip()
    Path("requirements.txt").write_text(requirements + "\n", encoding="utf-8")

    mas = """
# --- Build Stage ---
FROM python:3.11-slim as builder
WORKDIR /app
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    && rm -rf /var/lib/apt/lists/*
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
COPY . .

FROM python:3.11-slim
WORKDIR /app
RUN apt-get update && apt-get install -y --no-install-recommends \
    curl \
    && rm -rf /var/lib/apt/lists/*
COPY --from=builder /usr/local/lib/python3.11/site-packages /usr/local/lib/python3.11/site-packages
COPY --from=builder /app /app
EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
    """

    unity = """
FROM nvidia/opengl:1.2-glvnd-runtime-ubuntu20.04
WORKDIR /app
RUN apt-get update && apt-get install -y --no-install-recommends \
    libnss3 \
    libxss1 \
    libasound2 \
    libglu1-mesa \
    libxi6 \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*
COPY artifacts/unity/LinuxServer/ /app/
RUN chmod +x /app/SPARC-P.x86_64
CMD ["/app/SPARC-P.x86_64", "-logFile", "/dev/stdout", "-batchmode", "-force-vulkan"]
    """

    signaling = """
FROM node:20-alpine
WORKDIR /app
COPY artifacts/signaling/ /app/
RUN npm ci --omit=dev
EXPOSE 8080 8888
CMD ["node", "server.js", "--httpPort", "8080"]
    """

    with open("Dockerfile.mas", "w", encoding="utf-8") as f:
        f.write(mas.strip())
    with open("Dockerfile.unity-server", "w", encoding="utf-8") as f:
        f.write(unity.strip())
    with open("Dockerfile.signaling", "w", encoding="utf-8") as f:
        f.write(signaling.strip())

    missing = []
    if not unity_build_dir.exists():
        missing.append(str(unity_build_dir))
    if not signaling_dir.exists():
        missing.append(str(signaling_dir))
    if missing:
        print(f"Missing build artifacts before podman build: {missing}")
    else:
        print("Container build artifacts validated.")

create_dockerfiles()